In [124]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

### Post process the data

In [125]:
df = pd.read_csv('processedData.csv')
df.loc[:,'dx'] = df['x'].diff().fillna(0.0) 
df.loc[:,'dy'] = df['y'].diff().fillna(0.0)
df['accX'] = df.accX.ffill().add(df.accX.bfill()).div(2)
df['accY'] = df.accY.ffill().add(df.accY.bfill()).div(2)
df['accZ'] = df.accZ.ffill().add(df.accZ.bfill()).div(2)
df['gyrX'] = df.gyrX.ffill().add(df.gyrX.bfill()).div(2)
df['gyrY'] = df.gyrY.ffill().add(df.gyrY.bfill()).div(2)
df['gyrZ'] = df.gyrZ.ffill().add(df.gyrZ.bfill()).div(2) 
train, test = train_test_split(df, test_size=0.2)
train.head()

,Unnamed: 0,Unnamed: 0.1,time,accX,accY,accZ,gyrX,gyrY,gyrZ,y,x,dx,dy
98,100,100,1.982814,-0.57,-0.29,9.86,-0.061087,-0.020726,-0.003272,-0.416975,0.041489,0.000275,-0.005128
167,170,170,3.361653,0.01,0.00,9.74,-0.029452,-0.025089,-0.015272,-0.779103,0.064920,0.000559,-0.005250
182,186,186,3.680650,0.31,-0.23,9.46,0.005454,-0.035997,-0.006545,-0.863205,0.075875,0.000855,-0.005571
113,115,115,2.278714,0.03,-0.22,9.49,-0.026180,-0.002182,0.007636,-0.491875,0.045274,0.000278,-0.005640
159,161,161,3.191146,0.22,-0.15,9.97,0.021817,0.046906,-0.033816,-0.732674,0.060374,0.000516,-0.005856


In [126]:
Xtrain = np.column_stack([train["accX"], train["accY"], train["accZ"], train["gyrX"], train["gyrY"], train["gyrZ"]])#.T
Ytrain = np.column_stack([train["dx"], train["dy"]])#.T
Xtest = np.column_stack([test["accX"], test["accY"], test["accZ"], test["gyrX"], test["gyrY"], test["gyrZ"]])#.T
Ytest = np.column_stack([test["dx"], test["dy"]])#.T

In [127]:
nX = np.shape(Xtrain)[1]
nH1 = 24
nH2 = 16
nY = np.shape(Ytrain)[1]
nX

6

In [128]:
class NN(nn.Module):
    
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        #self.fc3 = nn.Linear(hidden1_size, num_classes)  
           
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [129]:
model = NN(nX, nH1, nH2, nY)
model = model.float()

### Convert data into a torch dataset

In [130]:
tensor_Xtrain = torch.from_numpy(Xtrain)
tensor_Ytrain = torch.from_numpy(Ytrain)
tensor_Xtest = torch.from_numpy(Xtest)
tensor_Ytest = torch.from_numpy(Ytest)
train_dataset = torch.utils.data.TensorDataset(tensor_Xtrain, tensor_Ytrain)
test_dataset = torch.utils.data.TensorDataset(tensor_Xtest, tensor_Ytest)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=5, shuffle=True)
# for data in train_dataset:
#     print(data)
#test_loader  = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Cost function and optimizer

In [131]:
cost = nn.MSELoss()
alpha = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=alpha, nesterov=True, momentum=0.9, dampening=0)  

### Train

In [132]:
episodes = 1000

for epis in range(episodes):
    for data in train_dataset:
        X, y = data
        model.zero_grad()
        output = model(X.float())#.view(-1, 6))
        loss = cost(output, y.float())
        loss.backward()
        optimizer.step()
    print(loss)

tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(1.7866e-06, grad_fn=<MseLossBackward>)
tensor(5.1605e-07, grad_fn=<MseLossBackward>)
tensor(4.8327e-07, grad_fn=<MseLossBackward>)
tensor(4.8266e-07, grad_fn=<MseLossBackward>)
tensor(4.8173e-07, grad_fn=<MseLossBackward>)
tensor(4.8065e-07, grad_fn=<MseLossBackward>)
tensor(4.7953e-07, grad_fn=<MseLossBackward>)
tensor(4.7844e-07, grad_fn=<MseLossBackward>)
tensor(4.7733e-07, grad_fn=<MseLossBackward>)
tensor(4.7623e-07, grad_fn=<MseLossBackward>)
tensor(4.7517e-07, grad_fn=<MseLossBackward>)
tensor(4.7410e-07, grad_fn=<MseLossBackward>)
tensor(4.7304e-07, grad_fn=<MseLossBackward>)
tensor(4.7198e-07, grad_fn=<MseLossBackward>)
tensor(4.7094e-07, grad_fn=<MseLossBackward>)
tensor(4.6991e-07, grad_fn=<MseLossBackward>)
tensor(4.6888e-07, grad_fn=<MseLossBackward>)
tensor(4.6786e-07, grad_fn=<MseLossBackward>)
tensor(4.6684e-07, grad_fn=<MseLossBackward>)
tensor(4.6584e-07, grad_fn=<MseLossBackward>)
tensor(4.6484e-07, grad_fn=<MseLossBac